## CSE595 Homework 1 : Logistic Regression Implementation

In [3]:
import pandas
import re

In [26]:
# Function to load stop words from NLTK Github Repo to be removed 
def load_stopwords():
    f = open("stopwords.txt")
    words = f.readlines()
    stopwords = set()
    for i in words:
        i = i.strip()
        stopwords.add(i)
    return stopwords

### Part 1 : Representing Text Data

In [4]:
# tokenizes purely by using whitespace
def tokenize (sentence : str):
    tokens = sentence.split()
    return tokens

In [ ]:
# tokenizing by ignoring capitalization, removing stop words as well as all punctuations
def better_tokenize(text : str):
    # store unique tokens in tokens
    tokens = set()
    # convert entire text into lowercase
    lowercase = text.lower()
    # find all possible words and all possible punctuations as their own tokens 
    words = re.findall(r"\w+|[,.!?#\r\n$%;:()\"']", lowercase)
    # load in stop words
    stopwords = load_stopwords()
    
    for word in words:
        if word not in stopwords:
            tokens.add(word)
    return tokens

#### Testing Tokenization

In [55]:
df = pandas.read_csv("train.csv")
better_tokenize(df['generation'][0])

{'"',
 "'",
 ',',
 '.',
 '10',
 'add',
 'always',
 'atmosphere',
 'back',
 'based',
 'certainly',
 'come',
 'coming',
 'conjuring',
 'disturb',
 'doesn',
 'even',
 'events',
 'every',
 'film',
 'films',
 'first',
 'full',
 'fun',
 'give',
 'good',
 'horror',
 'interesting',
 'jump',
 'least',
 'like',
 'long',
 'love',
 'makes',
 'movie',
 'one',
 'portion',
 'quite',
 'rating',
 'recent',
 'rewatch',
 'scares',
 'scariest',
 'since',
 'sinister',
 'star',
 'stick',
 'thrilling',
 'time',
 'times',
 'true',
 'watch',
 'watched',
 'watcher'}